In [1]:
import pandas as pd
import numpy as np
import torch
import sys

In [2]:
import torch_geometric
from torch_geometric.data import Data
from torch_geometric.data import DataLoader

sys.path.append('/home/jiaqing/桌面/Fea2Fea/property_process/')
from graph_property import G_property, binning

/home/jiaqing/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/least_angle.py:35: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  eps=np.finfo(np.float).eps,
/home/jiaqing/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/least_angle.py:597: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  eps=np.finfo(np.float).eps, copy_X=

In [3]:
property_file = pd.read_csv('geometric_graph_800_property.txt', sep = '\t')
edge_idx_file = pd.read_csv('geometric_graph_800_edge_idx.txt', sep = ',',header = None)

x = torch.tensor(np.array(property_file), dtype=torch.float)
edge_idx = torch.tensor(np.array(edge_idx_file), dtype=torch.long)

x_train = x[:,2].reshape((len(x),1))
tmp = np.array(x[:,1])
y = binning(tmp, k = 2,data_len = len(x))
#y = torch.tensor([[1] for i in range(200)])
#print(edge_idx)
data = Data(x=x_train, edge_index=edge_idx.t().contiguous(), y =y)
#print(data.y)
#print(data.x)

In [14]:
import random
num_nodes = len(data.y)
print(num_nodes)
num_train_nodes = int(num_nodes * 0.8)
num_valid_nodes = int(num_nodes * 0.1)
num_test_nodes = int(num_nodes * 0.1)
random.seed(1)
torch.manual_seed(1)
torch.cuda.manual_seed(1)
np.random.seed(1)
perm = torch.randperm(num_nodes)
train_idx = perm[:num_train_nodes]
valid_idx = perm[num_train_nodes:(num_train_nodes+num_valid_nodes)]
test_dix = perm[(num_train_nodes+num_valid_nodes):(num_train_nodes+num_valid_nodes+num_test_nodes)]

## debugging
print(train_idx)
print(valid_idx)

800
tensor([645, 435, 734, 593, 135,  63, 103, 269, 151, 467, 198, 170, 776, 696,
        453, 625, 134, 383, 290, 643, 350, 587, 308, 288, 355, 501, 420, 247,
        761, 153,  58, 396, 503, 200, 589, 518,   4, 766, 136, 400, 765, 445,
         43, 206, 215, 665, 746, 470, 465,  45, 164, 217, 276, 338, 229,  35,
        799, 614, 717,  22,  66, 775, 353, 186, 437, 473, 219,  73, 504, 638,
        469,  51,  84, 568, 432, 373, 157, 797, 509, 729, 191, 519, 560,  77,
        415, 385,  81,  54, 714, 104, 640, 513, 393, 534, 389, 497, 439, 199,
        172, 185, 253, 790, 491, 556,  23,  42, 268, 391, 139, 546, 783, 179,
        377,  53, 633, 617,  55, 419,  74, 117, 392, 363, 331, 654, 621, 154,
        351, 190,  68, 709, 649, 595, 536, 284, 324, 618, 252, 539,   9, 671,
        551, 102, 118,  39, 764,  97,  46, 774, 604, 552,  24, 598, 133, 737,
        706, 395, 101, 262, 267, 399, 695, 314, 597, 169, 349, 644, 570, 424,
         16, 121, 306, 620, 786, 700,  19, 233, 530,  52, 31

In [48]:
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.nn import GCNConv

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(1, 256)
        self.conv2 = GCNConv(256,64)
        
        self.linear1 = nn.Linear(64,16)
        self.linear2 = nn.Linear(16, 6)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = F.relu(self.linear1(x))
        x = self.linear2(x)

        return F.log_softmax(x, dim=1)

In [49]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.03, weight_decay=5e-4)



def train(data, train_idx):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[train_idx], data.y[train_idx])
    #print(loss)
    loss.backward()
    optimizer.step()
    
def test(data, valid_idx):
    model.eval()
    with torch.no_grad():
        pred = model(data).max(dim=1)[1]
    correct = 0
    correct += pred[valid_idx].eq(data.y[valid_idx]).sum().item()
    valid_acc = correct / len(valid_idx)
    return valid_acc

In [59]:
for epoch in range(400):
    train(data,train_idx)

a = test(data,test_dix)
print(a)

0.7625
